In [ ]:
%config InlineBackend.figure_format = 'retina'
import os

import ROOT
import numpy as np
from root_numpy import root2array, rec2array

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc

from array import array
from matplotlib.colors import LogNorm

from matplotlib import pyplot as plt
from event_weights import event_weights 
import seaborn as sns

np.seterr(divide='ignore', invalid='ignore')
plt.style.use('cmsstyle')

In [ ]:
min_values = [[-1., -120., -140., 0., 0., 0., 0.],
[-1., -140., -140., 0., 0., 0., 0.],
[-1., -170., -140., 0., 0., 0., 0.],
[-1., -220., -140., 0., 0., 0., 0.],
[-1., -60., -100., 0., 0., 0., 0.],
[-1., -70., -170., 0., 0., 0., 0.],
[-1., -70., -170., 0., 0., 0., 0.],
[-1., -70., -170., 0., 0., 0., 0., 0.]]

max_values = [[1., 120., 140, 3., 5., 12, 5.],
[1., 140., 140, 3., 5., 12., 5.],
[1., 170., 140, 3., 5., 12., 5.],
[1., 220., 140, 3., 5., 12., 5.],
[1., 60., 100., 3., 5., 12., 5.],
[1., 70., 170, 3., 5., 12., 5.],
[1., 70., 170, 3., 5., 12., 5.],
[1., 70., 170, 3., 5., 12., 5., 2.5]]

In [ ]:
# create DNN model

hidden1_neurons = 60
hidden2_neurons = 50

# Define Sequential model with 3 layers
model = keras.Sequential(
   [
       layers.Dense(65, input_dim=57, name="hidden_layer_1"),
       layers.Dropout(rate=0.2),
       layers.Dense(hidden1_neurons, activation="relu", name="hidden_layer_2"),
       layers.Dropout(rate=0.2),
       layers.Dense(hidden2_neurons, activation="relu", name="hidden_layer_3"),
       layers.Dense(1, activation="sigmoid", name="output_layer"),
   ]
)

model.layers
#---------
# DNN
#---------
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
inputfile = '/eos/user/b/bjoshi/RunIITau23Mu/AnalysisTrees/MuonPionTree_Preselected_Tracker.root'
rootfile = ROOT.TFile(inputfile,'READ')
tree_ = rootfile.Get('tree')
nevents = tree_.GetEntriesFast()

In [ ]:
n_signal_events = 0
n_bkg_events = 0

#Fill dataset for training
X = []
pt_array = []
eta_array = []
seg_comp_array = []
event_weights_array = []
y = []

# define weights
del weights
weights = event_weights()
weights.read_weights('/eos/user/b/bjoshi/RunIITau23Mu/event_weights/preselected/event_weights_tracker_muon.root')

for i in range(nevents):
    if (i%100000==0): print("Processing %d/%d" % (i, nevents))
    tree_.GetEntry(i)
    if (tree_.Muon_Pt<1.2 or tree_.Muon_Pt>=25 or abs(tree_.Muon_Eta)>2.4 or tree_.Muon_IsGlobalMuon==1 or tree_.Muon_IsPFMuon==0 or tree_.Muon_IsTrackerMuon==0): continue

    if (abs(tree_.Muon_pdgId)==13 and abs(tree_.Muon_motherPdgId)==15):
        y.append(1)
        n_signal_events += 1
    elif (abs(tree_.Muon_pdgId)==211 or abs(tree_.Muon_pdgId)==321):
        y.append(0)
        n_bkg_events += 1
    else: continue
    
    station_was_crossed = [0, 0, 0, 0, 0, 0, 0, 0]
    station_with_segment = [0, 0, 0, 0, 0, 0, 0, 0]
    input_array = [[0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0]]

    pt_array.append(tree_.Muon_Pt)
    eta_array.append(tree_.Muon_Eta)
    seg_comp_array.append(tree_.Muon_segmentCompatibility)
    tmp_weight = weights.get_weight(tree_.Muon_Pt, tree_.Muon_Eta)
    
    if (abs(tree_.Muon_pdgId)==13 and abs(tree_.Muon_motherPdgId)==15):
        event_weights_array.append(1.0)
    elif (abs(tree_.Muon_pdgId)==211 or abs(tree_.Muon_pdgId)==321):
        event_weights_array.append(tmp_weight)
        
    for iseg in range(8):
        station_id = iseg+1
        # status
        input_array[station_id-1][0] = getattr(tree_, 'Muon_station'+str(station_id)+'_status')
        # trackX
        input_array[station_id-1][1] = getattr(tree_, 'Muon_station'+str(station_id)+'_TrackX')
        # trackX
        input_array[station_id-1][2] = getattr(tree_, 'Muon_station'+str(station_id)+'_TrackY')
        # pull(dX)
        if (abs(getattr(tree_, 'Muon_station'+str(station_id)+'_pullX'))<99999.0):
            station_with_segment[station_id-1] = 1.0
            tmp_pullX = abs(getattr(tree_, 'Muon_station'+str(station_id)+'_pullX'))
            input_array[station_id-1][3] = tmp_pullX
        else: input_array[station_id-1][3] = max_values[station_id-1][3]
        # pull(dY)
        if (abs(getattr(tree_, 'Muon_station'+str(station_id)+'_pullY'))<99999.0):
            tmp_pullY = abs(getattr(tree_, 'Muon_station'+str(station_id)+'_pullY'))
            input_array[station_id-1][4] = tmp_pullY
        else: input_array[station_id-1][4] = max_values[station_id-1][4]
        # pull(dDxDz)
        if (abs(getattr(tree_, 'Muon_station'+str(station_id)+'_pullDxDz'))<99999.0):
            tmp_pullDxDz = abs(getattr(tree_, 'Muon_station'+str(station_id)+'_pullDxDz'))
            input_array[station_id-1][5] = tmp_pullDxDz
        else: input_array[station_id-1][5] = max_values[station_id-1][5]
        # pull(dDyDz)
        if (abs(getattr(tree_, 'Muon_station'+str(station_id)+'_pullDyDz'))<99999.0):
            tmp_pullDyDz = abs(getattr(tree_, 'Muon_station'+str(station_id)+'_pullDyDz'))
            input_array[station_id-1][6] = tmp_pullDyDz
        else: input_array[station_id-1][6] = max_values[station_id-1][6]

    X.append(input_array)

# Preprocess data
for x in X:
    for i in range(8):
        for j in range(1,7):
            norm = (max_values[i][j]-min_values[i][j])
            x[i][j] = (x[i][j]-min_values[i][j])/norm
            if (x[i][j]>1.0): x[i][j]=1.0
            if (x[i][j]<0.0): x[i][j]=0.0
                
for j in range(len(eta_array)):
    eta_array[j] = eta_array[j]/2.5

X = np.concatenate((np.reshape(X, (len(X), 56)), np.reshape(eta_array, (len(eta_array), 1))), axis=1)

In [ ]:
# generate training and testing samples
#X_train, X_test, y_train, y_test, seg_comp_train, seg_comp_test, event_weights_train, event_weights_test = train_test_split(X, np.reshape(y, (len(y),1)),  np.reshape(seg_comp_array,(len(y),1)), np.reshape(event_weights_array, (len(y),1)), test_size=0.30)
#model.fit(X_train, y_train, epochs=5, batch_size=100, sample_weight=event_weights_train[:,0])

# Print accuracy
#_, accuracy = model.evaluate(X_test, y_test)
#print('Accuracy: %.2f' % (accuracy*100))

# Compute ROC curve and area under the curve
#decisions = model.predict(X_test)
plot_title = 'roc_curve_comparison_hidden65_hidden_%d_hidden_%d_batch50_epoch10_reweighed_full.png' % (hidden1_neurons, hidden2_neurons)
compare_roc_segcomp_dnn(decisions, y_test, seg_comp_test, event_weights_test, plot_title)


#plot_input_variables(X_train, y_train, event_weights_train)
# serialize model to JSON
model_json = model.to_json()
with open(PLOT_DIR+"model_tracker.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save(PLOT_DIR+"model_tracker.h5")

In [ ]:
PLOT_DIR = './plots/'

In [ ]:
plot_title = 'compare_train_test_hidden65_hidden_%d_hidden_%d_batch50_epoch10_reweighed_full.png' % (hidden1_neurons, hidden2_neurons)
# Generate performace and prediction plots
compare_train_test(model, 'dnn', X_train, y_train, X_test, y_test, event_weights_train, event_weights_test, plot_title, bins=50)

In [ ]:
plt.figure(figsize=(6.6,6.6))
plt.hist(np.array(seg_comp_test)[np.array(y_test)==0], bins=50, range=(0,1.0), label='background', color='red', alpha=0.3, density=1)
plt.hist(np.array(seg_comp_test)[np.array(y_test)==1], bins=50, range=(0,1.0), label='signal', color='blue', alpha=0.3, density=1)
plt.ylabel("Arbitrary units")
plt.xlabel("CMSSW Segment Comp.")
plt.legend(loc='upper center', fontsize=14 )
#plt.show()
plt.savefig(PLOT_DIR+'/CMSSW_Segment_Compatibility.png', format='png')

In [ ]:
plt.figure(figsize=(6.6,6.6))
plt.hist(np.array(decisions)[np.logical_and(np.array(y_test)==0, np.array(seg_comp_test)<0.4)], bins=50, range=(0,1.0), label='background', color='red', alpha=0.3, density=1)
plt.hist(np.array(decisions)[np.logical_and(np.array(y_test)==1, np.array(seg_comp_test)<0.4)], bins=50, range=(0,1.0), label='signal', color='blue', alpha=0.3, density=1)
plt.ylabel("Arbitrary units")
plt.xlabel("DNN")
plt.legend(loc='upper center', fontsize=14 )
plt.show()

In [ ]:
plt.figure(figsize=(6.6,6.2))
H, xedges, yedges = np.histogram2d(np.array(seg_comp_test)[np.array(y_test)==0],np.array(decisions)[np.array(y_test)==0], bins=100, density=True)
# Mask zeros
Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
plt.pcolormesh(xedges,yedges,Hmasked)
cbar = plt.colorbar()
plt.ylabel("CMSSW Segment Comp.")
plt.xlabel("DNN score")
#plt.legend(loc='upper center', fontsize=14 )
plt.show()

In [ ]:
plt.figure(figsize=(6.6,6.2))
H, xedges, yedges = np.histogram2d(np.array(seg_comp_test)[np.array(y_test)==1],np.array(decisions)[np.array(y_test)==1], bins=100, density=True)
# Mask zeros
Hmasked = np.ma.masked_where(H==0,H) # Mask pixels with a value of zero
plt.pcolormesh(xedges,yedges,Hmasked)
cbar = plt.colorbar()
plt.ylabel("CMSSW Segment Comp.")
plt.xlabel("DNN score")
#plt.legend(loc='upper center', fontsize=14 )
plt.show()

In [ ]:
# function to compare train and test predictions
def compare_train_test(model, model_type, X_train, y_train, X_test, y_test, event_weights_train, event_weights_test, plot_name, bins=30):
    plt.clf()
    plt.figure(figsize=(6.6,6.6))
    decisions = [[], [], [], []]
    weights = [[],[],[],[]]
    if (model_type=='dnn'):
        predictions_train = model.predict(X_train)
        predictions_test = model.predict(X_test)
    elif (model_type=='bdt'):
        predictions_train = model.decision_function(X_train)
        predictions_test = model.decision_function(X_test)

    for j in range(len(X_train)):
        if(y_train[j]>0.5):
            decisions[0].append(predictions_train[j].item())
            weights[0].append(event_weights_train[j].item())
        else:
            decisions[1].append(predictions_train[j].item())
            weights[1].append(event_weights_train[j].item())

    for j in range(len(X_test)):
        if(y_test[j]>0.5):
            weights[2].append(event_weights_test[j].item())
            decisions[2].append(predictions_test[j].item())
        else:
            weights[3].append(event_weights_test[j].item())
            decisions[3].append(predictions_test[j].item())

    low = min(np.min(d) for d in decisions)
    high = max(np.max(d) for d in decisions)
    low_high = (low,high)

    plt.hist(decisions[0],
             color='b', alpha=0.5, range=low_high, bins=bins,
             histtype='stepfilled', density=True,
             label='S (train)', weights=weights[0])
    plt.hist(decisions[1],
             color='r', alpha=0.5, range=low_high, bins=bins,
             histtype='stepfilled', density = True,
             label='B (train)', weights=weights[1])

    hist, bins = np.histogram(decisions[2],
                              bins=bins, range=low_high, density=True, weights=weights[2])
    scale = len(decisions[2]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    width = (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.errorbar(center, hist, yerr=err, fmt='o', c='b', label='S (test)')

    hist, bins = np.histogram(decisions[3],
                              bins=bins, range=low_high, density=True, weights=weights[3])
    scale = len(decisions[2]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    plt.errorbar(center, hist, yerr=err, fmt='o', c='r', label='B (test)')
    plt.title('Overtraining check', fontsize=20)
    
    if (model_type=='dnn'): plt.xlabel("NN output")
    elif (model_type=='bdt'): plt.xlabel("BDT output")
    plt.ylabel("Arbitrary units")
    plt.legend(loc='upper center', fontsize=14 )
    #plt.show()
    plt.savefig(PLOT_DIR+model_type+'_'+plot_name, format='png')

In [ ]:
# function to generate ROC curve
def get_roc_curve(decisions, y_test, event_weights_test,  plot_name):
    plt.clf()
    plt.figure(figsize=(6.6,6.6))
    fpr, tpr, thresholds = roc_curve(y_test, decisions, sample_weight=event_weights_test)
    roc_auc = auc(fpr, tpr)

    plt.plot(tpr, 1-fpr, lw=1, label='ROC (area = %0.2f)'%(roc_auc))

    plt.plot([1, 0], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Random')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Background rejection')
    plt.ylabel('Signal acceptance')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower left", fontsize=14)
    plt.grid()
    #plt.show()
    plt.savefig(PLOT_DIR+plot_name, format='png')

In [ ]:
# compare ROC curves
def compare_roc_segcomp_dnn(decisions, y_test, segm_comp_test, event_weights_test, plot_name):
    plt.clf()
    plt.figure(figsize=(6.6,6.6))
    fpr, tpr, thresholds = roc_curve(y_test, decisions, sample_weight=event_weights_test)
    roc_auc = auc(fpr, tpr)

    plt.plot(tpr, 1-fpr, lw=1, label='ROC (area = %0.2f)'%(roc_auc))

    fpr, tpr, thresholds = roc_curve(y_test, seg_comp_test)
    roc_auc = auc(fpr, tpr)
    plt.plot(tpr, 1-fpr, lw=1, label='segment comp (area = %0.2f)'%(roc_auc), color='red')

    plt.plot([1, 0], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Random')
    plt.xlim([-0.0, 1.0])
    plt.ylim([-0.0, 1.0])
    plt.xlabel('Background rejection')
    plt.ylabel('Signal acceptance')
    plt.title('Receiver operating characteristic', fontsize=20)
    plt.legend(loc="lower left", fontsize=14)
    plt.grid()
    plt.savefig(PLOT_DIR+plot_name, format='png')